## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [306]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt
import matplotlib.image as mpimg
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [307]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None 

In [366]:
def calibrateCamera():
    
    global mtx, dist
    
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')
    
    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        img_size = img.shape[1:]

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

    ret, mtx, dist, rvec, tvec = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
    
    #cv2.imwrite('undist.jpg', cv2.undistort(cv2.imread('camera_cal/calibration1.jpg'), mtx, dist, None, mtx))
    
    return


In [334]:
def mag_threshold(image, sobel_kernel=3, thresh=(0, 255)):
    
    gry = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    sx = cv2.Sobel(gry, cv2.CV_64F, 1, 0, sobel_kernel)
    sy = cv2.Sobel(gry, cv2.CV_64F, 0, 1, sobel_kernel)
    
    sd = np.sqrt(np.square(sx) + np.square(sy))
    
    scaled_sd = np.uint8(255 * sd/np.max(sd))
    
    bmask = np.zeros_like(scaled_sd)
    
    bmask[np.logical_and((scaled_sd>thresh[0]), (scaled_sd<thresh[1]))] = 1
    
    return bmask

def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):

    gry = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    sx = cv2.Sobel(gry, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sy = cv2.Sobel(gry, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    asx = np.absolute(sx)
    asy = np.absolute(sy)
    
    ang = np.arctan2(asy, asx)
    
    dir_binary = np.zeros(ang.shape)
    
    dir_binary[np.logical_and((ang > thresh[0]), (ang < thresh[1]))] = 1
    
    return dir_binary

def color_threshold(img, thresh=(0, 255)):

    hls = cv2.cvtColor(img,cv2.COLOR_BGR2HLS)
    
    s  = hls[:,:,2]
    color_binary = np.zeros(s.shape)
    color_binary[(s>thresh[0]) & (s<=thresh[1])] = 1
    
    return color_binary

In [335]:
def edge_detect(goodimg):
    
    if line_l.detected == False or line_r.detected == False:
        
        mag_binary = mag_threshold(goodimg, sobel_kernel=7, thresh=(30,100))
        #dir_binary = dir_threshold(goodimg, sobel_kernel=27, thresh=(0.7, 1.3))
        color_binary = color_threshold(goodimg, (100, 250))
                
    else:
        # Optimization
        window_margin = 100
        y = np.array(range(0,goodimg.shape[0],window_margin))
        
        # Find the left and right lane x-coordinates from current fit
        x_l = line_l.current_fit[0] * y **2 + line_l.current_fit[1] * y + line_l.current_fit[2]
        x_r = line_r.current_fit[0] * y **2 + line_r.current_fit[1] * y + line_r.current_fit[2]

        # Adjust the x coordinates to be within the picture frame
        x_l = np.int32(np.maximum(x_l, window_margin * np.ones(len(x_l))))
        x_r = np.int32(np.minimum(x_r, (goodimg.shape[1] - window_margin) * np.ones(len(x_r))))
        
        # Apply magnitude thresholding to 200-by-100 pixel rectangles on the current lanes only
        mag_lst_l = [mag_threshold(goodimg[y[i]:y[i+1], x_l[i]-window_margin : x_l[i]+window_margin, :], sobel_kernel=7, thresh=(30,100)) for i in range(len(y)-1)]
        mag_lst_r = [mag_threshold(goodimg[y[i]:y[i+1], x_r[i]-window_margin : x_r[i]+window_margin, :], sobel_kernel=7, thresh=(30,100)) for i in range(len(y)-1)]

        # Now do the same for color thresholding
        color_lst_l = [color_threshold(goodimg[y[i]:y[i+1], x_l[i]-window_margin : x_l[i]+window_margin, :], (100, 250)) for i in range(len(y)-1)]
        color_lst_r = [color_threshold(goodimg[y[i]:y[i+1], x_r[i]-window_margin : x_r[i]+window_margin, :], (100, 250)) for i in range(len(y)-1)]

        mag_binary = np.zeros((goodimg.shape[0], goodimg.shape[1]), dtype=int)
        color_binary = np.zeros_like(mag_binary)
        
        # Build a binary frame from thresholding results
        for i in range(len(y)-1):
            mag_binary[y[i]:y[i+1], x_l[i]-window_margin : x_l[i]+window_margin] = mag_lst_l[i]
            mag_binary[y[i]:y[i+1], x_r[i]-window_margin : x_r[i]+window_margin] = mag_lst_r[i]
            
            color_binary[y[i]:y[i+1], x_l[i]-window_margin : x_l[i]+window_margin] = color_lst_l[i]
            color_binary[y[i]:y[i+1], x_r[i]-window_margin : x_r[i]+window_margin] = color_lst_r[i]
        
    # Make a composite binary frame
    fin_bin = np.zeros_like(mag_binary)
    fin_bin[(mag_binary == 1) | (color_binary == 1)] = 1
    
    return fin_bin

In [336]:
# Perspective Transform
def perspective_transform():
    global M, invM
    
    src = np.float32([[520, 470], [770, 470], [1200, 720], [200, 720]])
    dst = np.float32([[200,0], [1200,0], [1200, 720], [200, 720]])
    M = cv2.getPerspectiveTransform(src, dst)
    invM = cv2.getPerspectiveTransform(dst, src)
    
    return

In [375]:
def find_lane_pixels(binary_warped):
    
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    #out_img = np.zeros_like(binary_warped)

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(binary_warped.shape[1]//2)

    if line_l.detected == False or line_r.detected == False:
        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    else:
        # If lanes already found, use it as starting point
        leftx_base = int(line_l.bestx[-1])
        rightx_base = int(line_r.bestx[-1])
        
        #print(leftx_base, rightx_base)

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    good_left_inds = (nonzerox<0)
    good_right_inds = (nonzerox<0)
    
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        left_idx = (nonzerox >= win_xleft_low)&(nonzerox < win_xleft_high)&(nonzerox < midpoint)&(nonzeroy >= win_y_low) & (nonzeroy < win_y_high)
        good_left_inds = good_left_inds | left_idx

        right_idx = (nonzerox >= win_xright_low)&(nonzerox < win_xright_high)&(nonzerox >= midpoint) & (nonzeroy >= win_y_low) & (nonzeroy < win_y_high)  
        good_right_inds = good_right_inds | right_idx
        

        #print(good_left_inds, good_left_inds)
        
        # Append these indices to the lists
        #left_lane_inds.append(good_left_inds)
        #right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if(np.count_nonzero(nonzerox[left_idx]) > minpix): # Remove this when you add your function
            leftx_current = int(np.mean(nonzerox[left_idx]))
        if(np.count_nonzero(nonzerox[right_idx]) > minpix):
            rightx_current = int(np.mean(nonzerox[right_idx]))

    
    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    #try:
    #    left_lane_inds = np.concatenate(left_lane_inds)
    #    right_lane_inds = np.concatenate(right_lane_inds)
    #except ValueError:
    #    # Avoids an error if the above is not implemented fully
    #    pass

    # Extract left and right line pixel positions
    leftx = nonzerox[good_left_inds]
    lefty = nonzeroy[good_left_inds] 
    rightx = nonzerox[good_right_inds]
    righty = nonzeroy[good_right_inds]

    line_l.allx = leftx
    line_l.ally = lefty
    line_r.allx = rightx
    line_r.ally = righty
    
    return out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    out_img = find_lane_pixels(binary_warped)
    

    ### Fit a second order polynomial to each using `np.polyfit` ###

    left_fit = np.polyfit(line_l.ally, line_l.allx, 2)
    line_l.recent_xfitted.append(left_fit)
    line_l.current_fit = left_fit

    if len(line_l.recent_xfitted) > 10:
        line_l.recent_xfitted.pop(0)

    left_fit_coeff = np.array(line_l.recent_xfitted)    
    line_l.best_fit = np.mean(left_fit_coeff, axis=0)
    
    right_fit = np.polyfit(line_r.ally, line_r.allx, 2)
    line_r.recent_xfitted.append(right_fit)
    line_r.current_fit = right_fit

    if len(line_r.recent_xfitted) > 10:
        line_r.recent_xfitted.pop(0)

    right_fit_coeff = np.array(line_r.recent_xfitted)
    line_r.best_fit = np.mean(right_fit_coeff, axis=0)

    # Generate x and y values for plotting
    ploty = np.linspace(0, out_img.shape[0]-1, out_img.shape[0])
    
    try:
        ### Calc both polynomials using ploty, left_fit and right_fit ###
        line_l.bestx = line_l.best_fit[0] * ploty ** 2 + line_l.best_fit[1] * ploty + line_l.best_fit[2]
        line_r.bestx = line_r.best_fit[0] * ploty ** 2 + line_r.best_fit[1] * ploty + line_r.best_fit[2]
        
        line_l.detected = True
        line_r.detected = True
        
    except TypeError:
        line_l.detected = False
        line_r.detected = False
        
    ## Visualization ##
    # Colors in the left and right lane regions
    #out_img[line_l.ally, line_l.allx] = [255, 0, 0]
    #out_img[line_r.ally, line_r.allx] = [0, 0, 255]
    
    # Plots the left and right polynomials on the lane lines
    #out_img[np.int32(ploty), np.int32(line_l.bestx)] = [255,255,0]
    #out_img[np.int32(ploty), np.int32(line_r.bestx)] = [255,255,0]
    #cv2.imwrite('fitting.jpg',out_img)
    
    return

In [338]:
def measure_curvature_real(y_eval):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym = 30/720 # meters per pixel in y dimension
    xm = 3.7/700 # meters per pixel in x dimension
    
    y_eval = ym * y_eval

    ##### Implement the calculation of R_curve (radius of curvature) #####
    line_l.radius_of_curvature  = ((1 + ((xm/ym**2)*2* line_l.best_fit[0] * y_eval + (xm/ym)*line_l.best_fit[1])**2) ** 1.5)/np.abs((xm/ym**2)*2*line_l.best_fit[0])
    line_r.radius_of_curvature  = ((1 + ((xm/ym**2)*2* line_r.best_fit[0] * y_eval + (xm/ym)*line_r.best_fit[1])**2) ** 1.5)/np.abs((xm/ym**2)*2*line_r.best_fit[0])
    
    return

In [339]:
def make_overlay(img_mask):
    
    global invM
    
    ploty = np.linspace(0, img_mask.shape[0]-1, img_mask.shape[0])
    
    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    
    pts = list((line_l.bestx[i], ploty[i]) for i in range(0,len(ploty)))
    lst = list((line_r.bestx[i], ploty[i]) for i in range(0,len(ploty)))
    lst.reverse()
    pts.extend(lst)
    
    v_arr = np.array([pts], dtype = np.int32)    

    cv2.fillPoly(img_mask, v_arr, (0,230))
    
    #cv2.imshow('mask',img_mask)
    
    warped = cv2.warpPerspective(img_mask, invM, (img_mask.shape[1], img_mask.shape[0]), flags=cv2.INTER_LINEAR)
    
    #cv2.imshow('overlay',warped)
    
    return warped

In [363]:
def find_lanes(img):
    global mtx, dist, M, invM
    
    #3
    goodimg = cv2.undistort(img, mtx, dist, None, mtx)
    
    #cv2.imwrite('undistort.jpg',img)

    #4
    
    if line_l.detected == True and line_r.detected == True:
        binary_warped = cv2.warpPerspective(goodimg, M, (goodimg.shape[1], goodimg.shape[0]), flags=cv2.INTER_LINEAR)
        binary_warped = edge_detect(binary_warped)
    else:
        fin_bin = edge_detect(goodimg)
        binary_warped = cv2.warpPerspective(fin_bin, M, (fin_bin.shape[1], fin_bin.shape[0]), flags=cv2.INTER_LINEAR)
        
        edge_img = np.dstack((fin_bin, fin_bin, fin_bin)) * 255
        #cv2.imwrite('binary.jpg',edge_img)
        
        prsptv_img = np.dstack((binary_warped, binary_warped, binary_warped)) * 255
        #cv2.imwrite('perspective.jpg',prsptv_img)
    
    #5
    fit_polynomial(binary_warped)

    #6
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = img.shape[0]

    measure_curvature_real(y_eval)

    #print(left_curverad, right_curverad)
    
    line_l.line_base_pos = line_l.bestx[-1] - img.shape[1]//2
    line_r.line_base_pos = line_r.bestx[-1] - img.shape[1]//2

    #7
    mask = np.zeros_like(img)
    overlay = make_overlay(mask)

    anot = cv2.addWeighted(img, 1, overlay, 0.3, 0)

    offset = (line_l.line_base_pos + line_r.line_base_pos) * (3.7/700) # converted from pixel space to real world
    curv = (line_l.radius_of_curvature + line_r.radius_of_curvature) / 2
    
    cv2.putText(anot, 'Radius of Curvature = {:^4} m'.format(int(curv)), (380, 30), 3, 1, (255,255,255))
    
    if offset > 0:
        cv2.putText(anot, 'Vehicle is {:^4.2} m left of center'.format(abs(offset)), (380, 70), 3, 1, (255,255,255))
    else:
        cv2.putText(anot, 'Vehicle is {:^4.2} m right of center'.format(abs(offset)), (380, 70), 3, 1, (255,255,255))
    
    return anot

In [341]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
        
    result = find_lanes(image)

    return result

In [373]:
def pipeline():
    global mtx, dist, M, invM
    
    #1
    calibrateCamera()

    #2
    perspective_transform()
    
    #img = cv2.imread('test_images/test1.jpg')
    #rslt = process_image(img)

    #cv2.imwrite('lanes.jpg',rslt)
    
    white_output = 'video_out.mp4'
    ## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
    ## To do so add .subclip(start_second,end_second) to the end of the line below
    ## Where start_second and end_second are integer values representing the start and end of the subclip
    ## You may also uncomment the following line for a subclip of the first 5 seconds
    #clip1 = VideoFileClip("project_video.mp4").subclip(0,5)
    clip1 = VideoFileClip("project_video.mp4")
    white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
    white_clip.write_videofile(white_output, audio=False)    

In [376]:
line_l = Line()
line_r = Line()

pipeline()

                                                                                                                       

                                                                                                                 
                                                                                                                    


t:   1%|▍                                                        | 1/125 [71:04:33<8813:25:09, 255873.47s/it, now=None]

t:   4%|██▋                                                             | 52/1260 [67:52:38<56:31,  2.81s/it, now=None]


                                                                                                                       

                                                                                                                 
                                                                                                                    


t:   1%|▍                                        

Moviepy - Building video video_out.mp4.
Moviepy - Writing video video_out.mp4







t:   0%|                                                                    | 2/1260 [00:00<02:33,  8.19it/s, now=None]



t:   0%|▏                                                                   | 4/1260 [00:00<02:16,  9.20it/s, now=None]



t:   0%|▎                                                                   | 6/1260 [00:00<02:08,  9.73it/s, now=None]



t:   1%|▍                                                                   | 8/1260 [00:00<02:01, 10.34it/s, now=None]



t:   1%|▌                                                                  | 10/1260 [00:00<01:53, 11.00it/s, now=None]



t:   1%|▋                                                                  | 12/1260 [00:01<01:49, 11.42it/s, now=None]



t:   1%|▋                                                                  | 14/1260 [00:01<01:47, 11.57it/s, now=None]



t:   1%|▊                                                                  | 16/1260 [00:01<01:46, 11.68it/s, now=None]



t:   1%|▉   

t:  19%|████████████▋                                                     | 242/1260 [00:20<01:28, 11.48it/s, now=None]



t:  19%|████████████▊                                                     | 244/1260 [00:21<01:27, 11.65it/s, now=None]



t:  20%|████████████▉                                                     | 246/1260 [00:21<01:25, 11.84it/s, now=None]



t:  20%|████████████▉                                                     | 248/1260 [00:21<01:28, 11.46it/s, now=None]



t:  20%|█████████████                                                     | 250/1260 [00:21<01:28, 11.44it/s, now=None]



t:  20%|█████████████▏                                                    | 252/1260 [00:21<01:28, 11.33it/s, now=None]



t:  20%|█████████████▎                                                    | 254/1260 [00:21<01:30, 11.14it/s, now=None]



t:  20%|█████████████▍                                                    | 256/1260 [00:22<01:28, 11.39it/s, now=None]



t:  20%|████████

t:  38%|█████████████████████████▏                                        | 482/1260 [00:41<01:06, 11.64it/s, now=None]



t:  38%|█████████████████████████▎                                        | 484/1260 [00:42<01:06, 11.75it/s, now=None]



t:  39%|█████████████████████████▍                                        | 486/1260 [00:42<01:07, 11.42it/s, now=None]



t:  39%|█████████████████████████▌                                        | 488/1260 [00:42<01:07, 11.47it/s, now=None]



t:  39%|█████████████████████████▋                                        | 490/1260 [00:42<01:05, 11.71it/s, now=None]



t:  39%|█████████████████████████▊                                        | 492/1260 [00:42<01:05, 11.76it/s, now=None]



t:  39%|█████████████████████████▉                                        | 494/1260 [00:42<01:05, 11.64it/s, now=None]



t:  39%|█████████████████████████▉                                        | 496/1260 [00:43<01:05, 11.59it/s, now=None]



t:  40%|████████

t:  57%|█████████████████████████████████████▌                            | 718/1260 [01:03<00:45, 11.88it/s, now=None]



t:  57%|█████████████████████████████████████▋                            | 720/1260 [01:03<00:46, 11.65it/s, now=None]



t:  57%|█████████████████████████████████████▊                            | 722/1260 [01:03<00:45, 11.75it/s, now=None]



t:  57%|█████████████████████████████████████▉                            | 724/1260 [01:03<00:45, 11.87it/s, now=None]



t:  58%|██████████████████████████████████████                            | 726/1260 [01:04<00:45, 11.76it/s, now=None]



t:  58%|██████████████████████████████████████▏                           | 728/1260 [01:04<00:45, 11.80it/s, now=None]



t:  58%|██████████████████████████████████████▏                           | 730/1260 [01:04<00:44, 11.92it/s, now=None]



t:  58%|██████████████████████████████████████▎                           | 732/1260 [01:04<00:45, 11.60it/s, now=None]



t:  58%|████████

t:  76%|██████████████████████████████████████████████████▏               | 958/1260 [01:24<00:26, 11.42it/s, now=None]



t:  76%|██████████████████████████████████████████████████▎               | 960/1260 [01:24<00:26, 11.45it/s, now=None]



t:  76%|██████████████████████████████████████████████████▍               | 962/1260 [01:24<00:25, 11.63it/s, now=None]



t:  77%|██████████████████████████████████████████████████▍               | 964/1260 [01:24<00:26, 11.38it/s, now=None]



t:  77%|██████████████████████████████████████████████████▌               | 966/1260 [01:24<00:26, 11.29it/s, now=None]



t:  77%|██████████████████████████████████████████████████▋               | 968/1260 [01:25<00:25, 11.34it/s, now=None]



t:  77%|██████████████████████████████████████████████████▊               | 970/1260 [01:25<00:26, 10.75it/s, now=None]



t:  77%|██████████████████████████████████████████████████▉               | 972/1260 [01:25<00:26, 10.99it/s, now=None]



t:  77%|████████

t:  93%|████████████████████████████████████████████████████████████▍    | 1172/1260 [01:44<00:07, 11.38it/s, now=None]



t:  93%|████████████████████████████████████████████████████████████▌    | 1174/1260 [01:44<00:07, 11.60it/s, now=None]



t:  93%|████████████████████████████████████████████████████████████▋    | 1176/1260 [01:44<00:07, 11.66it/s, now=None]



t:  93%|████████████████████████████████████████████████████████████▊    | 1178/1260 [01:44<00:06, 11.97it/s, now=None]



t:  94%|████████████████████████████████████████████████████████████▊    | 1180/1260 [01:44<00:06, 11.73it/s, now=None]



t:  94%|████████████████████████████████████████████████████████████▉    | 1182/1260 [01:45<00:06, 11.63it/s, now=None]



t:  94%|█████████████████████████████████████████████████████████████    | 1184/1260 [01:45<00:06, 11.46it/s, now=None]



t:  94%|█████████████████████████████████████████████████████████████▏   | 1186/1260 [01:45<00:06, 11.58it/s, now=None]



t:  94%|████████

Moviepy - Done !
Moviepy - video ready video_out.mp4
